In [64]:
import nltk
from nltk import CFG, PCFG
from nltk.parse import pchart
import sys, re

In [65]:
def strip_comment(s):
    """Remove comments from a string containing nltk CFG"""
    return re.sub(r'#.*?$', "", s, flags=re.MULTILINE)

In [66]:
grammar = CFG.fromstring("""
N -> N N | 'natural' | 'language' | 'learning' | 'course'
""")

inp = 'natural language learning course'
print("Start:", grammar.start(), file=sys.stderr)
print("Productions:", grammar.productions(), file=sys.stderr)
parser = nltk.ChartParser(grammar)
for tree in parser.parse(inp.split()):
    print(tree)

(N (N (N (N natural) (N language)) (N learning)) (N course))
(N (N (N natural) (N (N language) (N learning))) (N course))
(N (N natural) (N (N (N language) (N learning)) (N course)))
(N (N natural) (N (N language) (N (N learning) (N course))))
(N (N (N natural) (N language)) (N (N learning) (N course)))


Start: N
Productions: [N -> N N, N -> 'natural', N -> 'language', N -> 'learning', N -> 'course']


In [67]:
grammar_string = """
N -> N N [{}] 
N -> 'natural' [{}] 
N -> 'language' [{}] 
N -> 'learning' [{}]
N -> 'course' [{}]
""".format(1/5, 1/5, 1/5, 1/5, 1/5)
print(grammar_string)

# uses N -> N N three times and each of
# the words once each and because each
# rule has the same probability we can
# easily compute the probability of
# each parse tree for this PCFG
print((1/5)**3 * (1/5)**4)

grammar = PCFG.fromstring(grammar_string)

inp = 'natural language learning course'
print("Start:", grammar.start(), file=sys.stderr)
print("Productions:", grammar.productions(), file=sys.stderr)
parser = pchart.InsideChartParser(grammar)
for tree in parser.parse(inp.split()):
    print(tree)


N -> N N [0.2] 
N -> 'natural' [0.2] 
N -> 'language' [0.2] 
N -> 'learning' [0.2]
N -> 'course' [0.2]

1.2800000000000006e-05
(N
  (N natural)
  (N (N language) (N (N learning) (N course)))) (p=1.28e-05)
(N
  (N natural)
  (N (N (N language) (N learning)) (N course))) (p=1.28e-05)
(N
  (N (N natural) (N language))
  (N (N learning) (N course))) (p=1.28e-05)
(N
  (N (N natural) (N (N language) (N learning)))
  (N course)) (p=1.28e-05)
(N
  (N (N (N natural) (N language)) (N learning))
  (N course)) (p=1.28e-05)


Start: N
Productions: [N -> N N [0.2], N -> 'natural' [0.2], N -> 'language' [0.2], N -> 'learning' [0.2], N -> 'course' [0.2]]


In [68]:
grammar_string = """
N_1_4 -> N_1   N_2_4
N_1_4 -> N_1_3 N_4
N_1_4 -> N_1_2 N_3_4
N_1_2 -> N_1   N_2   # natural language
N_2_3 -> N_2   N_3   # language learning
N_3_4 -> N_3   N_4   # learning course
N_1_3 -> N_1_2 N_3   # natural language learning
N_1_3 -> N_1   N_2_3 # natural language learning
N_2_4 -> N_2_3 N_4   # language learning course
N_2_4 -> N_2   N_3_4 # language learning course
N_1 -> 'natural' 
N_2 -> 'language' 
N_3 -> 'learning'
N_4 -> 'course'
"""
print(strip_comment(grammar_string))
grammar = CFG.fromstring(strip_comment(grammar_string))
#print("Start:", grammar.start(), file=sys.stderr)
#print("Productions:", grammar.productions(), file=sys.stderr)
parser = nltk.ChartParser(grammar)
inp = 'natural language learning course'.strip()
for tree in parser.parse(inp.split()):
    print(tree)


N_1_4 -> N_1   N_2_4
N_1_4 -> N_1_3 N_4
N_1_4 -> N_1_2 N_3_4
N_1_2 -> N_1   N_2   
N_2_3 -> N_2   N_3   
N_3_4 -> N_3   N_4   
N_1_3 -> N_1_2 N_3   
N_1_3 -> N_1   N_2_3 
N_2_4 -> N_2_3 N_4   
N_2_4 -> N_2   N_3_4 
N_1 -> 'natural' 
N_2 -> 'language' 
N_3 -> 'learning'
N_4 -> 'course'

(N_1_4
  (N_1_3 (N_1_2 (N_1 natural) (N_2 language)) (N_3 learning))
  (N_4 course))
(N_1_4
  (N_1_3 (N_1 natural) (N_2_3 (N_2 language) (N_3 learning)))
  (N_4 course))
(N_1_4
  (N_1 natural)
  (N_2_4 (N_2_3 (N_2 language) (N_3 learning)) (N_4 course)))
(N_1_4
  (N_1 natural)
  (N_2_4 (N_2 language) (N_3_4 (N_3 learning) (N_4 course))))
(N_1_4
  (N_1_2 (N_1 natural) (N_2 language))
  (N_3_4 (N_3 learning) (N_4 course)))


In [69]:
# write down the probabilities as fractions over 
# the possible choices for each non-terminal in
# the lhs which has multiple possible rhs expansions
N_1_4_probs = [1/3, 1/3, 1/3]
N_1_3_probs = [1/2, 1/2]
N_2_4_probs = [1/2, 1/2]

grammar_string = """
N_1_4 -> N_1   N_2_4   [{}]
N_1_4 -> N_1_3 N_4     [{}]
N_1_4 -> N_1_2 N_3_4   [{}]
N_1_2 -> N_1   N_2     [1.0] # natural language
N_2_3 -> N_2   N_3     [1.0] # language learning
N_3_4 -> N_3   N_4     [1.0] # learning course
N_1_3 -> N_1_2 N_3     [{}]  # natural language learning
N_1_3 -> N_1   N_2_3   [{}]  # natural language learning
N_2_4 -> N_2_3 N_4     [{}]  # language learning course
N_2_4 -> N_2   N_3_4   [{}]  # language learning course
N_1 -> 'natural'       [1.0]
N_2 -> 'language'      [1.0]
N_3 -> 'learning'      [1.0]
N_4 -> 'course'        [1.0]
""".format(*N_1_4_probs, *N_1_3_probs, *N_2_4_probs) 
# using * the lists are flattened and we get (1/3, 1/3, 1/3, 1/2, 1/2, 1/2, 1/2)

print(strip_comment(grammar_string))
grammar = PCFG.fromstring(strip_comment(grammar_string))
print("Start:", grammar.start(), file=sys.stderr)
print("Productions:", grammar.productions(), file=sys.stderr)
parser = pchart.InsideChartParser(grammar)
parser.trace(3)
inp = 'natural language learning course'.strip()
for tree in parser.parse(inp.split()):
    print(tree)


N_1_4 -> N_1   N_2_4   [0.3333333333333333]
N_1_4 -> N_1_3 N_4     [0.3333333333333333]
N_1_4 -> N_1_2 N_3_4   [0.3333333333333333]
N_1_2 -> N_1   N_2     [1.0] 
N_2_3 -> N_2   N_3     [1.0] 
N_3_4 -> N_3   N_4     [1.0] 
N_1_3 -> N_1_2 N_3     [0.5]  
N_1_3 -> N_1   N_2_3   [0.5]  
N_2_4 -> N_2_3 N_4     [0.5]  
N_2_4 -> N_2   N_3_4   [0.5]  
N_1 -> 'natural'       [1.0]
N_2 -> 'language'      [1.0]
N_3 -> 'learning'      [1.0]
N_4 -> 'course'        [1.0]

  |[-] . . .| [0:1] 'natural'                        [1.0]
  |. [-] . .| [1:2] 'language'                       [1.0]
  |. . [-] .| [2:3] 'learning'                       [1.0]
  |. . . [-]| [3:4] 'course'                         [1.0]
  |. . . [-]| [3:4] 'course'                         [1.0]
  |. . . [-]| [3:4] N_4 -> 'course' *                [1.0]
  |. . . > .| [3:3] N_4 -> * 'course'                [1.0]
  |. . [-] .| [2:3] 'learning'                       [1.0]
  |. . [-] .| [2:3] N_3 -> 'learning' *              [1.0]
  |. 

Start: N_1_4
Productions: [N_1_4 -> N_1 N_2_4 [0.333333], N_1_4 -> N_1_3 N_4 [0.333333], N_1_4 -> N_1_2 N_3_4 [0.333333], N_1_2 -> N_1 N_2 [1.0], N_2_3 -> N_2 N_3 [1.0], N_3_4 -> N_3 N_4 [1.0], N_1_3 -> N_1_2 N_3 [0.5], N_1_3 -> N_1 N_2_3 [0.5], N_2_4 -> N_2_3 N_4 [0.5], N_2_4 -> N_2 N_3_4 [0.5], N_1 -> 'natural' [1.0], N_2 -> 'language' [1.0], N_3 -> 'learning' [1.0], N_4 -> 'course' [1.0]]


In [70]:
N_1_4_probs = [1/30, 28/30, 1/30]
N_1_3_probs = [19/20, 1/20]
N_2_4_probs = [1/2, 1/2]